## Models
- SMOTE
- ROSE
- One-class SVM, SGD one-class SVM
- Isolation forest
- Local outlier factor
- Elliptical envelope
- Logistic?
- RF?
- XGBoost?

In [1]:
import pickle
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


class Models:

    def __init__(self):
        for df in ['X_train', 'y_train']:
            with open(f'./pickles/{df}.obj', 'rb') as f:
                setattr(self, f'{df}', pickle.load(f))

models = Models()
smote = SMOTE()


In [3]:
X, y = smote.fit_resample(models.X_train, models.y_train)

In [8]:
y.value_counts()

-1    187141
 1    187141
Name: target, dtype: int64

In [ ]:
xgb_clf = XGBClassifier()
rndf_clf = RandomForestClassifier()
